In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

from flask import Flask, render_template, url_for, redirect, request, session
import pandas as pd
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime as dt
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError, Length
from flask_bcrypt import Bcrypt

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///web.db'
db = SQLAlchemy(app)


################################################################################################################

#DB



app.config['SECRET_KEY'] = 'thisisasecretkey'


bcrypt = Bcrypt(app)

db.init_app(app)

class Todo (db.Model):
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(200), nullable=False)
    date_created = db.Column(db.DateTime, default=dt.utcnow)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
    def __repr__(self):
        return '<Task %r>' % self.id

class User (db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(15), unique=True, nullable=False)
    email = db.Column(db.String(50), unique=True, nullable=False)
    password = db.Column(db.String(80), nullable=False)
    notes = db.relationship('Todo', backref='writer', lazy='dynamic')

    def __repr__(self):
        return '<User %r>' % self.id
    
db.create_all()    


class RegisterForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")
            
    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address already exists. Please choose a different one.")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")



################################################################################################################

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/inspirations")
def inspirations():
    return render_template('inspirations.html')

# @app.route("/books")
# def books():
        # lists = [{'Book': 'Self Love Poetry', 'Author': 'Melody Godfred', 'Review': '4.8/5.0', 'Price': '$13.49'},
        # {'Book': 'Unconditional',  'Author': 'Linda Villines', 'Review': '5.0/5.0', 'Price': '$12.99'},
        # {'Book': 'Good Vibes, Good Life',  'Author': 'Vex King', 'Review': '4.7/5.0', 'Price': '$13.49'}]
    
        ##df = pd.DataFrame(lists)
        ##df.to_csv('books.csv',mode='w',index=False)


        # return render_template('books.html', entries = lists)
        
@app.route("/instructions")
def instructions():
    return render_template('instructions.html')

@app.route("/about")
def about():
    return render_template('about.html')

################################################################################################################

#TO-DO TASKS

@app.route("/comm", methods=['POST', 'GET'])
def comm():
    if request.method == 'POST':
        # task_content = request.form['content']
        # new_task = Todo(content = request.form['content'])
        # new_task = Todo(content = task_content)
        
        try:
            db.session.add(new_task)
            db.session.commit()
            return redirect("/comm")
        except:
            return 'there is a problem adding your task'
        
    else:
        tasks = Todo.query.order_by(Todo.date_created).all()
        return render_template('comm.html', tasks=tasks)

################################################################################################################

# SIGNUP AND LOGIN


@app.route("/login", methods=["GET", "POST"])  
def login():
    form = LoginForm()

    return render_template('login.html', form=form)
    
    
    
@app.route('/join', methods=['GET','POST'])
def join():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('join.html', form=form)




################################################################################################################

# ERROR HANDLERS

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

# RUN
if __name__ == '__main__':
    app.run('localhost', 1124)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/sammy/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
/Users/sammy/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:1124/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2021 10:23:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 10:23:28] "GET /join HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 10:23:33] "POST /login HTTP/1.1" 200 -
